In [1]:
import os
import pandas as pd
from re import search

def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

yearPDFList=[]

masterDF=pd.DataFrame()
for companyFolder in os.scandir(): #goes into every company
    if companyFolder.is_dir():
        print(companyFolder)
        for yearFolder in os.scandir(companyFolder):#goes into every year
            #yearPDFList=[]
            if yearFolder.is_dir():
                print(yearFolder)
                yearPath=yearFolder.path
                yearPDFList=[]
                for fileName in os.scandir(yearFolder):
                    if fileName.is_dir():#creating a list of all PDFNames
                        for monthFolder in os.scandir(fileName):
                            if monthFolder.path.endswith(".pdf"):
                                yearPDFList.append(monthFolder)
                    #print(yearPDFList)
                for fileName in os.scandir(yearFolder):
                    if fileName.path.endswith(".xlsx"): #scans every excel file in every year
                        filePath=os.path.abspath(fileName)
                        df = pd.read_excel (fileName)
                        cusip6=fileName.name[:find_nth(fileName.name,"_",1)] #extracts cusip6
                        ticker=fileName.name[find_nth(fileName.name,"_",1)+1:find_nth(fileName.name,"_",2)]
                        df.columns = df.iloc[11] #sets column headings (top row) as line 11
                        df["ExcelPath"]=filePath #adding ExcelPath
                        df["cusip6"]=cusip6
                        df["Ticker"]=ticker
                        df_subset=df.loc[12:].copy()
                        #print(yearPDFList)
                        containsPDFList=[]
                        PDFPathList=[]
                        for DCN in df_subset["DCN"]:
                            containsIt=False
                            for PDFName in yearPDFList:
                                if str(DCN) in PDFName.name:
                                    containsPDFList.append(PDFName.name)
                                    PDFPathList.append(PDFName.path)
                                    containsIt= True
                            if containsIt== False:
                                containsPDFList.append("")
                                PDFPathList.append("")
                        #df_subset["PDFName"]=containsPDFList    
                        #df_subset["PDFPath"]=PDFPathList
                        df_subset["PDFName"]=pd.Series(containsPDFList)
                        df_subset["PDFPath"]=pd.Series(PDFPathList)
                        masterDF=masterDF.append(df_subset,ignore_index = True)

<DirEntry '.ipynb_checkpoints'>
<DirEntry '00206R_T'>
<DirEntry '12'>
<DirEntry '13'>
<DirEntry '14'>
<DirEntry '15'>
<DirEntry '16'>


In [2]:
masterDF.to_csv('SeriesTest.csv', index=False)